**Practical Artificial Intelligence with Swift**

In [ ]:
# Install TuriCreate
!pip install turicreate

In [ ]:
# Import packages
import os
import json
import requests
import numpy as np
import turicreate as tc

In [ ]:
# The categories we want to be able to distinguish
categories = [
              "apple", "banana", "bread", "broccoli", "cake", "carrot", "coffee cup",
              "cookie", "donut", "grapes", "hot dog", "ice cream", "lollipop", "mushroom",
              "peanut", "pear", "pineapple", "pizza", "potato", "sandwich", "steak", "strawberry",
              "watermelon"
]

In [ ]:
# Configure as required
this_directory = os.path.join(os.getcwd())
quickdraw_directory = this_directory + "/quickdraw"
bitmap_directory = quickdraw_directory + "/bitmap"
bitmap_sframe_path = quickdraw_directory + "/bitmaps.sframe"
output_model_filename = this_directory + "/DrawingModel"
training_samples = 10000

In [ ]:
# Make some folders to put training data in
def make_directory(path):
  try:
    os.makedirs(path)
  except OSError:
    if not os.path.isdir(path):
      raise

make_directory(quickdraw_directory)
make_directory(bitmap_directory)

In [ ]:
# Fetch some data
bitmap_url = (
    "https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap"
)

total_categories = len(categories)

for index, category in enumerate(categories):
  bitmap_filename = "/" + category + ".npy"

  with open(bitmap_directory + bitmap_filename, 'wb') as bitmap_file:
    bitmap_response = requests.get(bitmap_url + bitmap_filename)
    bitmap_file.write(bitmap_response.content)

    print("Downloaded %s drawing (category %d/%d)" % (category, index + 1, total_categories))

  random_state = np.random.RandomState(100)

Downloaded apple drawing (category 1/23)
Downloaded banana drawing (category 2/23)
Downloaded bread drawing (category 3/23)
Downloaded broccoli drawing (category 4/23)
Downloaded cake drawing (category 5/23)
Downloaded carrot drawing (category 6/23)
Downloaded coffee cup drawing (category 7/23)
Downloaded cookie drawing (category 8/23)
Downloaded donut drawing (category 9/23)
Downloaded grapes drawing (category 10/23)
Downloaded hot dog drawing (category 11/23)
Downloaded ice cream drawing (category 12/23)
Downloaded lollipop drawing (category 13/23)
Downloaded mushroom drawing (category 14/23)
Downloaded peanut drawing (category 15/23)
Downloaded pear drawing (category 16/23)
Downloaded pineapple drawing (category 17/23)
Downloaded pizza drawing (category 18/23)
Downloaded potato drawing (category 19/23)
Downloaded sandwich drawing (category 20/23)
Downloaded steak drawing (category 21/23)
Downloaded strawberry drawing (category 22/23)
Downloaded watermelon drawing (category 23/23)


In [ ]:
def get_bitmap_sframe():
  labels, drawings = [], []
  for category in categories:
    data = np.load(
        bitmap_directory + "/" + category + ".npy",
        allow_pickle=True
    )
    random_state.shuffle(data)
    sampled_data = data[:training_samples]
    transformed_data = sampled_data.reshape(
        sampled_data.shape[0], 28, 28, 1
    )

    for pixel_data in transformed_data:
      image = tc.Image(_image_data=np.invert(pixel_data).tobytes(),
                       _width=pixel_data.shape[1],
                       _height=pixel_data.shape[0],
                       _channels=pixel_data.shape[2],
                       _format_enum=2,
                       _image_data_size=pixel_data.size)
      drawings.append(image)
      labels.append(category)
    print("...%s bitmaps complete" % category)
  print("%d bitmaps with %d labels" % (len(drawings), len(labels)))
  return tc.SFrame({'drawing': drawings, 'label': labels})

In [ ]:
# Save intermediate bitmap SFrame to files
bitmap_sframe = get_bitmap_sframe()
bitmap_sframe.save(bitmap_sframe_path)
bitmap_sframe.explore()

...apple bitmaps complete
...banana bitmaps complete
...bread bitmaps complete
...broccoli bitmaps complete
...cake bitmaps complete
...carrot bitmaps complete
...coffee cup bitmaps complete
...cookie bitmaps complete
...donut bitmaps complete
...grapes bitmaps complete
...hot dog bitmaps complete
...ice cream bitmaps complete
...lollipop bitmaps complete
...mushroom bitmaps complete
...peanut bitmaps complete
...pear bitmaps complete
...pineapple bitmaps complete
...pizza bitmaps complete
...potato bitmaps complete
...sandwich bitmaps complete
...steak bitmaps complete
...strawberry bitmaps complete
...watermelon bitmaps complete
230000 bitmaps with 230000 labels


/usr/local/lib/python3.7/dist-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,drawing,label
0,,apple
1,,apple
2,,apple
3,,apple
4,,apple
5,,apple
6,,apple
7,,apple
8,,apple
9,,apple


In [ ]:
# Train the drawing classifier
bitmap_model = tc.drawing_classifier.create(bitmap_sframe, 'label', max_iterations=50)

Download completed: /var/tmp/model_cache/drawing_classifier_pre_trained_model_245_classes_v0.mlmodel


Automatically generating validation set by sampling about 10000 out of 230000 datapoints.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using CPU to create model.

+--------------+----------------+---------------------+--------------+

...

| 47           | 0.918271       | 0.855367            | 2h 2m        |

| 48           | 0.919417       | 0.8491              | 2h 4m        |

| 49           | 0.919867       | 0.849896            | 2h 7m        |

| 50           | 0.921167       | 0.848801            | 2h 10m       |

+--------------+----------------+---------------------+--------------+

In [10]:
 # Export it to CoreML format
 bitmap_model.export_coreml(output_model_filename + '.mlmodel')